In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [5]:
from langchain.llms import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time


llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")

In [6]:
loader = TextLoader('90coffe in hittan.txt')
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")


You have 1 document
You have 4458520 characters in that document


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

In [8]:
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")


Now you have 1746 documents that have an average of 2,929 characters (smaller pieces)


In [9]:
batch_size = 500  # Adjust this based on your needs and token sizes
batches = [docs[i:i + batch_size] for i in range(0, len(docs), batch_size)]

for batch in batches:
    embeddings = OpenAIEmbeddings()
    docsearch = FAISS.from_documents(batch, embeddings)
    time.sleep(60)

In [12]:
# embeddings = OpenAIEmbeddings()
# docsearch = FAISS.from_documents(docs, embeddings)

AttributeError: 'OpenAIEmbeddings' object has no attribute 'embed'

In [10]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [11]:
def create_prompt(question):
    prompt = f"Analyze restaurant data for Hittan Alriyadh. Focus: {question}. Limit the response to key insights."
    return prompt

In [12]:
specific_question = "Identify key features of the restaurants in 2022 based on customer reviews."
prompt = create_prompt(specific_question)


In [13]:
response = qa.run(prompt)  # Adjust max_tokens as needed

# Print the response
print(response)


Based on the customer reviews provided for the restaurant in Riyadh, the following key features and insights can be identified for the year 2022:

1. **Quality of Coffee and Desserts**: Multiple reviews praise the quality of the coffee and desserts offered at the restaurant. Customers have mentioned "excellent coffee," "delicious sweets," and specific mentions of a "red cake" being very tasty.

2. **Ambiance and Comfort**: The ambiance of the restaurant is frequently described as nice, beautiful, and comfortable. One reviewer specifically appreciated the attention to small details by the owner, and the seating is described as very comfortable.

3. **Customer Service**: There are mixed reviews regarding customer service. While some customers experienced great service and mentioned staff by name, thanking them for their service, others had negative experiences, describing the service as poor or the staff as rude.

4. **Variety**: The restaurant seems to offer a wide variety of coffee bea

In [ ]:
specific_question = "Identify key features of the restaurants in 2022 based on customer reviews."
prompt = create_prompt(specific_question)
response = qa.run(prompt)  # Adjust max_tokens as needed
print(response)

